In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    #model_name = "unsloth/mistral-7b-v0.3-bnb-4bit",

    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# ### Instruction:
# {}

alpaca_prompt = """Aşağıda bir görevi tanımlayan bir talimat ve daha fazla bağlam sağlayan bir giriş bulunmaktadır. Talebi uygun bir şekilde tamamlayan bir yanıt yaz.

### Input:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    #instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    #for instruction, input, output in zip(instructions, inputs, outputs):
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        #text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("RsGoksel/Finansal", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/1247 [00:00<?, ? examples/s]

Map:   0%|          | 0/1247 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1247 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
soru = """
Bir kripto para birimi olan "CryptoX", son bir ayda %30 değer kazandı. Bu süreçte, ABD Merkez Bankası (FED), faiz oranlarını beklenmedik bir şekilde artırdı ve bu da doların değer kazanmasına yol açtı. Aynı dönemde, Avrupa'daki ekonomik durgunluk nedeniyle Euro değer kaybetti ve altın fiyatlarında bir artış gözlendi.
Ancak, CryptoX'in arkasındaki teknolojiye olan güven son zamanlarda arttı ve büyük kurumsal yatırımcılar bu coini portföylerine eklemeye başladı. Öte yandan, bazı ülkeler kriptoya karşı regülasyonları sıkılaştırmayı planlıyor.

Soru:
CryptoX'in gelecekteki fiyat hareketleri hakkında bir tahminde bulunmak için hangi makroekonomik göstergeleri önceliklendirmelisiniz ve bu göstergeler arasında nasıl bir ilişki kurmalısınız? Bu analiz ışığında, CryptoX'in değerinin artmaya devam edeceğini mi, yoksa düşüşe geçeceğini mi öngörüyorsunuz?
"""

# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        #"Finansal alanda ilgili sorulara cevap ver. En az 300 kelime ile cevap ver.", # instruction
        "Likidite nedir",
        "",
        #soru, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 320)

In [ ]:
# Save to 8bit Q8_0
if 1: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive


Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
import shutil
shutil.copy('/content/model/unsloth.F16.gguf','/content/drive/MyDrive/HeisenMechLLM/')

'/content/drive/MyDrive/HeisenMechLLM/unsloth.F16.gguf'

In [ ]:
#!pip install gpt4all

from gpt4all import GPT4All

model = GPT4All("/content/drive/MyDrive/HeisenMech_Unsloth8/unsloth.Q8_0.gguf", device='cpu', allow_download=False)
#output = model.generate("Türkçe cevap ver, CP violation nedir, Detaylıca açıkla.", max_tokens=920, streaming=True)
system_prompt = '### System:\n Kullanıcılara finansal alanda yardım et..\n\n'
prompt_template = '### Kullanıcı:\n{0}\n\n### Cevap:\n'

soru = """
Bir kripto para birimi olan "CryptoX", son bir ayda %30 değer kazandı. Bu süreçte, ABD Merkez Bankası (FED), faiz oranlarını beklenmedik bir şekilde artırdı ve bu da doların değer kazanmasına yol açtı. Aynı dönemde, Avrupa'daki ekonomik durgunluk nedeniyle Euro değer kaybetti ve altın fiyatlarında bir artış gözlendi.
Ancak, CryptoX'in arkasındaki teknolojiye olan güven son zamanlarda arttı ve büyük kurumsal yatırımcılar bu coini portföylerine eklemeye başladı. Öte yandan, bazı ülkeler kriptoya karşı regülasyonları sıkılaştırmayı planlıyor.

Soru:
CryptoX'in gelecekteki fiyat hareketleri hakkında bir tahminde bulunmak için hangi makroekonomik göstergeleri önceliklendirmelisiniz ve bu göstergeler arasında nasıl bir ilişki kurmalısınız? Bu analiz ışığında, CryptoX'in değerinin artmaya devam edeceğini mi, yoksa düşüşe geçeceğini mi öngörüyorsunuz?
"""

tokens = []
with model.chat_session(system_prompt=system_prompt):
    for token in model.generate(soru, streaming=True, max_tokens=650):
        tokens.append(token)
        print(token,end="")